<a href="https://colab.research.google.com/github/kolomichyk/BMSTU-Hack/blob/develop/Diadal_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from datetime import datetime

In [5]:
data_train = pd.read_csv('train.csv')

In [ ]:
data_train.head(20)

In [ ]:
data_train.describe()

,wagnum,prev_fr_id,prev_is_load,prev_snd_org_id,prev_rsv_org_id,prev_distance,snd_st_id,rsv_st_id,fr_id,is_load,rod,common_ch,snd_org_id,rsv_org_id,distance
count,1.192243e+06,1.192243e+06,1.192243e+06,1.192243e+06,1.192243e+06,1.191209e+06,1.192243e+06,1.192243e+06,1.192243e+06,1.192243e+06,1192243.0,1.192243e+06,1.192243e+06,1.192243e+06,1.190917e+06
mean,3.817986e+06,3.810650e+06,5.375909e-01,3.734411e+06,4.983752e+06,1.559562e+03,5.061955e+06,4.974726e+06,3.948631e+06,5.826908e-01,8.0,2.904683e+02,3.905827e+06,4.902682e+06,1.544651e+03
std,2.202737e+06,3.123850e+06,4.985851e-01,2.185192e+06,2.615266e+06,1.763346e+03,2.636698e+06,2.587721e+06,3.244060e+06,4.931150e-01,0.0,4.388080e+02,2.327654e+06,2.481025e+06,1.738946e+03
min,1.100000e+01,8.384500e+04,0.000000e+00,1.224700e+04,4.190000e+02,0.000000e+00,3.693000e+03,3.693000e+03,6.336600e+04,0.000000e+00,8.0,9.000000e+00,7.639000e+03,4.190000e+02,0.000000e+00
25%,1.911908e+06,1.399685e+06,0.000000e+00,2.327070e+06,3.434232e+06,2.760000e+02,2.492408e+06,2.492408e+06,1.399685e+06,0.000000e+00,8.0,9.000000e+00,2.327070e+06,4.097118e+06,2.760000e+02
50%,3.823218e+06,1.661408e+06,1.000000e+00,2.327070e+06,4.597939e+06,1.052000e+03,5.326970e+06,5.440640e+06,1.959160e+06,1.000000e+00,8.0,9.000000e+00,2.327070e+06,4.272291e+06,1.005000e+03
75%,5.730952e+06,7.399715e+06,1.000000e+00,4.597939e+06,7.093343e+06,1.886000e+03,7.153615e+06,6.889687e+06,7.399715e+06,1.000000e+00,8.0,9.750000e+02,4.995013e+06,6.683733e+06,1.879000e+03
max,7.637731e+06,9.990826e+06,1.000000e+00,9.979008e+06,9.997469e+06,1.462700e+04,9.988340e+06,9.990165e+06,9.990826e+06,1.000000e+00,8.0,9.750000e+02,9.998667e+06,9.997469e+06,1.462700e+04


In [60]:
data_train.dtypes

wagnum                 int64
prev_fr_id             int64
prev_is_load         float64
prev_date_depart      object
prev_date_arrival     object
prev_snd_org_id        int64
prev_rsv_org_id        int64
prev_distance        float64
snd_st_id              int64
rsv_st_id              int64
fr_id                  int64
is_load                int64
date_depart           object
common_ch            float64
vidsobst              object
snd_org_id           float64
rsv_org_id           float64
distance             float64
prev_freight          object
prev_fr_group         object
freight               object
fr_group              object
Travel_time          float64
Mounth                 int64
dotw                   int64
dtype: object

In [ ]:
data_train.shape

(1192243, 22)

In [ ]:
data_train.isna().sum()

wagnum                  0
prev_fr_id              0
prev_is_load            0
prev_date_depart        4
prev_date_arrival     197
prev_snd_org_id         0
prev_rsv_org_id         0
prev_distance        1034
snd_st_id               0
rsv_st_id               0
fr_id                   0
is_load                 0
date_depart             0
rod                     0
common_ch               0
vidsobst                0
snd_org_id              0
rsv_org_id              0
distance             1326
prev_freight            1
prev_fr_group           1
freight                 4
fr_group                4
dtype: int64

In [6]:
data_train = data_train.drop(columns=['rod'])

In [69]:
data_train.head()

,wagnum,prev_fr_id,prev_is_load,prev_date_depart,prev_date_arrival,prev_snd_org_id,prev_rsv_org_id,prev_distance,snd_st_id,rsv_st_id,...,snd_org_id,rsv_org_id,distance,prev_freight,prev_fr_group,freight,fr_group,Travel_time,Mounth,dotw
0,11,1399685,0.0,2022-03-30 22:55:00,2022-04-05 11:07:00,4266126,2279057,4493.0,3609555,4985724,...,2279057.0,9321847.0,269.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,БАНКЕТКИ,Остальные грузы,132.200000,0,0
1,11,292757,1.0,2022-04-07 08:45:00,2022-04-07 18:22:00,2279057,9321847,269.0,4985724,2492408,...,9321847.0,4272291.0,3584.0,БАНКЕТКИ,Остальные грузы,ГАРНИТУРЫ КУХОН,Остальные грузы,9.616667,0,0
2,11,8674472,1.0,2022-04-11 07:30:00,2022-04-20 15:15:00,9321847,4272291,3584.0,2492408,4288665,...,4272291.0,2555202.0,417.0,ГАРНИТУРЫ КУХОН,Остальные грузы,КАРНИЗЫ ДЕРЕВ,Лесные грузы,223.750000,0,0
3,11,8210522,1.0,2022-04-23 16:20:00,2022-04-28 03:43:00,4272291,2555202,417.0,4288665,2440270,...,2327070.0,2327070.0,248.0,КАРНИЗЫ ДЕРЕВ,Лесные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,107.383333,0,0
4,11,1399685,0.0,2022-05-03 14:47:00,2022-05-04 05:11:00,2327070,2327070,248.0,2440270,2440270,...,2327070.0,2487325.0,5.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,ВАТА МЕД,Остальные грузы,14.400000,0,0


In [43]:
def GetHours(date):
    date_object = datetime.strptime(date[:10], '%Y-%m-%d')
    data_seconds_arr = list(date[11:].split(':'))
    data_low = int(data_seconds_arr[0]) + int(data_seconds_arr[1])/60 + int(data_seconds_arr[2])/3600
    data_high = int(date_object.timestamp())
    return data_high/3600 + data_low

In [72]:
def ConvertDate(data):
    for i in range(data.shape[0]):
        if (type(data['prev_date_depart'][i]) != float and type(data['prev_date_arrival'][i]) != float):
          depart = GetHours(data['prev_date_depart'][i])
          arrival = GetHours(data['prev_date_arrival'][i])
          data['Travel_time'][i] = arrival - depart

In [67]:
def Check(data):
  mass = list()
  flag = False
  for i in range(data.shape[0]):
    if (type(data['prev_date_depart'][i]) == float and not flag):
      flag = True
      print(data['prev_date_depart'][i])
    mass.append(type(data['prev_date_depart'][i]))
  return mass

In [79]:
def AddDotw(data):
  for i in range(data.shape[0]):
    if (type(data['prev_date_arrival'][i]) != float):
      date_tmp = datetime.strptime(data['prev_date_arrival'][i][:10], '%Y-%m-%d')
      data['dotw'][i] = datetime.isoweekday(date_tmp)

In [90]:
def AddMounth(data):
  for i in range(data.shape[0]):
    if (type(data['prev_date_arrival'][i]) != float):
      tmp = data['prev_date_arrival'][i][:10]
      tmp = list(tmp.split('-'))
      data['Mounth'][i] = int(tmp[1])

In [45]:
data_train['Travel_time'] = [0 for i in range(data_train.shape[0])]
data_train['Mounth'] = [0 for i in range(data_train.shape[0])]
data_train['dotw'] = [0 for i in range(data_train.shape[0])]

In [92]:
data_train.head(10)

,wagnum,prev_fr_id,prev_is_load,prev_date_depart,prev_date_arrival,prev_snd_org_id,prev_rsv_org_id,prev_distance,snd_st_id,rsv_st_id,...,snd_org_id,rsv_org_id,distance,prev_freight,prev_fr_group,freight,fr_group,Travel_time,Mounth,dotw
0,11,1399685,0.0,2022-03-30 22:55:00,2022-04-05 11:07:00,4266126,2279057,4493.0,3609555,4985724,...,2279057.0,9321847.0,269.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,БАНКЕТКИ,Остальные грузы,132.200000,4,2
1,11,292757,1.0,2022-04-07 08:45:00,2022-04-07 18:22:00,2279057,9321847,269.0,4985724,2492408,...,9321847.0,4272291.0,3584.0,БАНКЕТКИ,Остальные грузы,ГАРНИТУРЫ КУХОН,Остальные грузы,9.616667,4,4
2,11,8674472,1.0,2022-04-11 07:30:00,2022-04-20 15:15:00,9321847,4272291,3584.0,2492408,4288665,...,4272291.0,2555202.0,417.0,ГАРНИТУРЫ КУХОН,Остальные грузы,КАРНИЗЫ ДЕРЕВ,Лесные грузы,223.750000,4,3
3,11,8210522,1.0,2022-04-23 16:20:00,2022-04-28 03:43:00,4272291,2555202,417.0,4288665,2440270,...,2327070.0,2327070.0,248.0,КАРНИЗЫ ДЕРЕВ,Лесные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,107.383333,4,4
4,11,1399685,0.0,2022-05-03 14:47:00,2022-05-04 05:11:00,2327070,2327070,248.0,2440270,2440270,...,2327070.0,2487325.0,5.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,ВАТА МЕД,Остальные грузы,14.400000,5,3
5,11,9276341,0.0,2022-05-05 23:02:22,2022-05-05 23:45:42,2327070,2487325,5.0,2440270,2440270,...,2327070.0,7980401.0,5.0,ВАТА МЕД,Остальные грузы,ВАТА МЕД,Остальные грузы,0.722222,5,4
6,11,9276341,0.0,2022-05-06 00:52:38,2022-05-06 06:03:42,2327070,7980401,5.0,2440270,8450101,...,7980401.0,6550555.0,67.0,ВАТА МЕД,Остальные грузы,РИС НЕШЕЛУШ,Хлебные грузы,5.184444,5,5
7,11,2309180,1.0,2022-05-09 00:13:00,2022-05-10 00:54:00,7980401,6550555,67.0,8450101,2440270,...,2327070.0,2327070.0,67.0,РИС НЕШЕЛУШ,Хлебные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,24.683333,5,2
8,11,1399685,0.0,2022-05-13 07:30:00,2022-05-14 07:54:00,2327070,2327070,67.0,2440270,8450101,...,7980401.0,6550555.0,67.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,РИС НЕШЕЛУШ,Хлебные грузы,24.400000,5,6
9,11,2309180,1.0,2022-05-20 15:06:00,2022-05-21 16:57:00,7980401,6550555,67.0,8450101,2440270,...,2327070.0,7980401.0,67.0,РИС НЕШЕЛУШ,Хлебные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,25.850000,5,6


In [73]:
ConvertDate(data_train)

In [93]:
AddDotw(data_train)

<ipython-input-79-5ee6d3c92915>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dotw'][i] = datetime.isoweekday(date_tmp)


In [91]:
AddMounth(data_train)

<ipython-input-90-abec1bc088b6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Mounth'][i] = int(tmp[1])


In [15]:
data_train.head()

(596381, 25)

In [68]:
print(set(Check(data_train)))

nan
{<class 'str'>, <class 'float'>}


In [94]:
data_train.shape[0]

596381